![ARM](../img/05.png)

# ARM

## Objective
Discover interesting patterns and relationships between drugs, medical conditions, and side effects using association rule mining techniques.

## Contents
1. **Data Preparation for Market Basket Analysis**
2. **Frequent Itemset Mining**
3. **Association Rule Generation**
4. **Drug-Condition Association Analysis**
5. **Side Effect Pattern Discovery**
6. **Drug Interaction and Co-occurrence Analysis**
7. **Business Insights and Recommendations**

## Business Applications
- **Drug Recommendation Systems**: Suggest alternative medications based on condition patterns
- **Side Effect Prediction**: Identify likely side effects based on drug combinations
- **Treatment Pathway Analysis**: Discover common medication sequences for conditions
- **Pharmaceutical Market Analysis**: Understand drug usage patterns and market relationships

## Techniques Used
- **Apriori Algorithm**: For frequent itemset mining
- **FP-Growth**: Alternative frequent pattern mining
- **Association Rules**: Confidence, support, and lift metrics
- **Market Basket Analysis**: Transaction-based pattern discovery

---


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from pathlib import Path
import pickle
from collections import Counter
import itertools
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

# Association rule mining libraries
try:
    from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
    from mlxtend.preprocessing import TransactionEncoder
    print(" MLxtend successfully imported")
except ImportError:
    print(" MLxtend not found. Installing...")
    import subprocess
    subprocess.check_call(["pip", "install", "mlxtend"])
    from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
    from mlxtend.preprocessing import TransactionEncoder
    print(" MLxtend installed and imported")

# Network analysis for drug relationships
try:
    import networkx as nx
    print(" NetworkX available for relationship graphs")
except ImportError:
    print(" NetworkX not available, will use basic visualizations")
    nx = None

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Set up pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

# Robust Plotly display function (reusing from EDA)
def show_plotly_figure(fig, title=""):
    """Robust function to display Plotly figures with multiple fallback options"""
    try:
        fig.show()
        return True
    except Exception as e:
        print(f" Plotly rendering failed: {e}")
        print(f" Figure '{title}' data is ready but couldn't be displayed")
        return False

# Set up Plotly error handling
original_show = go.Figure.show
def robust_show(self, *args, **kwargs):
    try:
        return original_show(self, *args, **kwargs)
    except Exception as e:
        print(f" Plotly rendering failed: {e}")
        return None
go.Figure.show = robust_show

print(" Association Rule Mining Environment Ready!")
print(" Libraries loaded with robust error handling")
print(" Ready to discover hidden patterns in drug data!")


 MLxtend successfully imported
 NetworkX available for relationship graphs
 Association Rule Mining Environment Ready!
 Libraries loaded with robust error handling
 Ready to discover hidden patterns in drug data!


---
## 1. Data Loading and Preparation


In [2]:
# Load the processed dataset
df = pd.read_csv("../data/drugs_processed.csv")
print(f" Dataset loaded: {df.shape}")
print(f" Columns available: {len(df.columns)}")

# Load metadata if available
try:
    with open("../data/feature_engineering_metadata.pkl", "rb") as f:
        metadata = pickle.load(f)
    print(f" Metadata loaded - Features added: {metadata.get('features_added', 'N/A')}")
except FileNotFoundError:
    print(" Metadata file not found, proceeding without metadata")
    metadata = {}

# Display basic information
print(f"\n  Dataset Summary:")
print(f"    Total records: {len(df):,}")
print(f"    Unique conditions: {df['medical_condition'].nunique()}")
print(f"    Unique drugs: {df['drug_name'].nunique()}")
print(f"    Rating range: {df['rating'].min():.1f} - {df['rating'].max():.1f}")

# Check for key columns needed for association rule mining
required_columns = ['drug_name', 'medical_condition', 'side_effects']
available_columns = [col for col in required_columns if col in df.columns]
print(f"\n Required columns available: {available_columns}")

# Show sample data structure
print(f"\n Sample data:")
display_cols = ['drug_name', 'medical_condition', 'rating', 'side_effects']
available_display_cols = [col for col in display_cols if col in df.columns]
print(df[available_display_cols].head(3).to_string())


 Dataset loaded: (2931, 84)
 Columns available: 84
 Metadata loaded - Features added: 67

  Dataset Summary:
    Total records: 2,931
    Unique conditions: 47
    Unique drugs: 2912
    Rating range: 0.0 - 10.0

 Required columns available: ['drug_name', 'medical_condition', 'side_effects']

 Sample data:
        drug_name medical_condition  rating                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

---
## 2. Data Preparation for Market Basket Analysis


In [3]:
# Data preparation for association rule mining
print(" PREPARING DATA FOR MARKET BASKET ANALYSIS")
print("=" * 50)

# 1. Prepare Drug-Condition Transactions
print("\n 1.Creating Drug-Condition Transaction Data")

# Create transactions where each medical condition is a basket containing drugs
condition_drug_transactions = []
for condition in df['medical_condition'].unique():
    condition_drugs = df[df['medical_condition'] == condition]['drug_name'].unique().tolist()
    if len(condition_drugs) > 1:  # Only include conditions with multiple drugs
        condition_drug_transactions.append(condition_drugs)

print(f"    Total condition-based transactions: {len(condition_drug_transactions)}")
print(f"    Average drugs per condition: {np.mean([len(t) for t in condition_drug_transactions]):.1f}")

# 2. Prepare Side Effects Transactions
print("\n 2.Creating Side Effects Transaction Data")

# Parse side effects if they exist
side_effect_transactions = []
if 'side_effects' in df.columns:
    for idx, row in df.iterrows():
        if pd.notna(row['side_effects']) and len(str(row['side_effects'])) > 10:
            # Extract side effects (assuming they're comma-separated)
            side_effects = str(row['side_effects']).lower().split(',')
            side_effects = [effect.strip() for effect in side_effects if len(effect.strip()) > 2]
            if len(side_effects) > 1:
                side_effect_transactions.append(side_effects)

print(f"    Total side effect transactions: {len(side_effect_transactions)}")
if side_effect_transactions:
    print(f"    Average side effects per transaction: {np.mean([len(t) for t in side_effect_transactions]):.1f}")

# 3. Prepare Drug-Rating Category Transactions
print("\n 3.Creating Drug-Rating Category Transaction Data")

# Create rating categories
df['rating_category'] = pd.cut(df['rating'], 
                              bins=[0, 4, 6, 8, 10], 
                              labels=['Low_Rating', 'Medium_Rating', 'High_Rating', 'Excellent_Rating'])

# Create transactions combining drugs with their rating categories
drug_rating_transactions = []
for condition in df['medical_condition'].unique():
    condition_data = df[df['medical_condition'] == condition]
    transaction = []
    
    for _, row in condition_data.iterrows():
        # Add drug name
        transaction.append(f"Drug_{row['drug_name']}")
        # Add rating category
        transaction.append(f"Rating_{row['rating_category']}")
    
    if len(set(transaction)) > 2:  # Only include if there are multiple unique items
        drug_rating_transactions.append(list(set(transaction)))

print(f"   Total drug-rating transactions: {len(drug_rating_transactions)}")

# 4. Create Binary Matrix for Drug-Condition Analysis
print("\n 4️.Creating Binary Matrix for Drug-Condition Analysis")

# Create a transaction encoder for drug-condition data
te = TransactionEncoder()
te_ary = te.fit(condition_drug_transactions).transform(condition_drug_transactions)
drug_condition_df = pd.DataFrame(te_ary, columns=te.columns_)

print(f"    Binary matrix shape: {drug_condition_df.shape}")
print(f"    Unique drugs in matrix: {len(te.columns_)}")
print(f"    Sparsity: {(1 - drug_condition_df.sum().sum() / drug_condition_df.size) * 100:.1f}%")

# 5. Create Summary Statistics
print("\n Transaction Summary Statistics:")

# Drug frequency analysis
drug_frequency = {}
for transaction in condition_drug_transactions:
    for drug in transaction:
        drug_frequency[drug] = drug_frequency.get(drug, 0) + 1

# Sort by frequency
top_drugs = sorted(drug_frequency.items(), key=lambda x: x[1], reverse=True)[:10]

print(f"\n Top 10 Most Frequent Drugs Across Conditions:")
for i, (drug, freq) in enumerate(top_drugs, 1):
    print(f"   {i:2d}. {drug}: appears in {freq} conditions")

# Condition complexity analysis
condition_complexity = df.groupby('medical_condition')['drug_name'].nunique().sort_values(ascending=False)
print(f"\n Top 5 Conditions with Most Drug Options:")
for i, (condition, drug_count) in enumerate(condition_complexity.head().items(), 1):
    print(f"   {i}. {condition}: {drug_count} different drugs")

# Store prepared data for use in subsequent analyses
print(f"\n  Data preparation completed!")
print(f"    Ready for association rule mining")
print(f"    Multiple transaction formats prepared")
print(f"    Binary matrices created for analysis")


 PREPARING DATA FOR MARKET BASKET ANALYSIS

 1.Creating Drug-Condition Transaction Data
    Total condition-based transactions: 47
    Average drugs per condition: 62.3

 2.Creating Side Effects Transaction Data
    Total side effect transactions: 2928
    Average side effects per transaction: 23.4

 3.Creating Drug-Rating Category Transaction Data
   Total drug-rating transactions: 47

 4️.Creating Binary Matrix for Drug-Condition Analysis
    Binary matrix shape: (47, 2912)
    Unique drugs in matrix: 2912
    Sparsity: 97.9%

 Transaction Summary Statistics:

 Top 10 Most Frequent Drugs Across Conditions:
    1. triamcinolone: appears in 3 conditions
    2. clindamycin: appears in 2 conditions
    3. erythromycin: appears in 2 conditions
    4. doxepin: appears in 2 conditions
    5. ciclesonide: appears in 2 conditions
    6. beclomethasone: appears in 2 conditions
    7. diphenhydramine: appears in 2 conditions
    8. mometasone: appears in 2 conditions
    9. hydrocortisone: appe

---
## 3. Frequent Itemset Mining and Association Rules


In [4]:
# Frequent Itemset Mining and Association Rule Generation
print(" FREQUENT ITEMSET MINING & ASSOCIATION RULE GENERATION")
print("=" * 60)

# 1. Apply Apriori Algorithm for Drug-Condition Associations
print("\n 1. Mining Frequent Drug Combinations (Apriori Algorithm)")

# Set minimum support threshold (adjustable based on dataset size)
min_support = max(0.01, 3/len(condition_drug_transactions))  # At least 3 occurrences or 1%
print(f"    Minimum support threshold: {min_support:.3f}")

try:
    # Apply Apriori algorithm
    frequent_itemsets = apriori(drug_condition_df, min_support=min_support, use_colnames=True)
    print(f"    Found {len(frequent_itemsets)} frequent itemsets")
    
    if len(frequent_itemsets) > 0:
        # Display top frequent itemsets
        frequent_itemsets_sorted = frequent_itemsets.sort_values('support', ascending=False)
        print(f"\n Top 10 Frequent Drug Combinations:")
        for i, (idx, row) in enumerate(frequent_itemsets_sorted.head(10).iterrows(), 1):
            itemset = list(row['itemsets'])
            if len(itemset) > 1:  # Only show combinations, not single items
                print(f"   {i:2d}. {itemset} (support: {row['support']:.3f})")
    
    else:
        print("    No frequent itemsets found - trying lower support threshold")
        min_support = max(0.005, 2/len(condition_drug_transactions))
        frequent_itemsets = apriori(drug_condition_df, min_support=min_support, use_colnames=True)
        print(f"    Found {len(frequent_itemsets)} frequent itemsets with support ≥ {min_support:.3f}")

except Exception as e:
    print(f"    Apriori algorithm failed: {e}")
    print("    This might be due to sparse data - will try alternative approach")
    frequent_itemsets = pd.DataFrame()

# 2. Generate Association Rules
print("\n 2. Generating Association Rules")

association_rules_df = pd.DataFrame()
if len(frequent_itemsets) > 0:
    try:
        # Generate association rules with different metrics
        association_rules_df = association_rules(frequent_itemsets, 
                                                metric="confidence", 
                                                min_threshold=0.1)
        
        if len(association_rules_df) > 0:
            print(f"    Generated {len(association_rules_df)} association rules")
            
            # Add rule descriptions
            association_rules_df['rule_description'] = (
                association_rules_df['antecedents'].astype(str) + 
                " → " + 
                association_rules_df['consequents'].astype(str)
            )
            
            # Sort by lift (most interesting rules first)
            association_rules_df = association_rules_df.sort_values('lift', ascending=False)
            
            print(f"\n Top 10 Association Rules (by Lift):")
            print(f"{'Rule':<50} {'Support':<8} {'Conf':<6} {'Lift':<6}")
            print("-" * 70)
            
            for i, (idx, row) in enumerate(association_rules_df.head(10).iterrows(), 1):
                antecedents = list(row['antecedents'])[0] if len(row['antecedents']) == 1 else str(list(row['antecedents']))
                consequents = list(row['consequents'])[0] if len(row['consequents']) == 1 else str(list(row['consequents']))
                rule = f"{antecedents} → {consequents}"
                
                print(f"{rule[:48]:<50} {row['support']:.3f}    {row['confidence']:.3f}  {row['lift']:.2f}")
        
        else:
            print("    No association rules met the minimum confidence threshold")
    
    except Exception as e:
        print(f"    Association rule generation failed: {e}")

# 3. Alternative Analysis: Direct Drug Co-occurrence
print("\n 3. Direct Drug Co-occurrence Analysis")

# Calculate drug co-occurrence matrix
drug_cooccurrence = {}
for transaction in condition_drug_transactions:
    for i, drug1 in enumerate(transaction):
        for drug2 in transaction[i+1:]:
            pair = tuple(sorted([drug1, drug2]))
            drug_cooccurrence[pair] = drug_cooccurrence.get(pair, 0) + 1

# Convert to DataFrame for easier analysis
if drug_cooccurrence:
    cooccurrence_data = []
    total_transactions = len(condition_drug_transactions)
    
    for (drug1, drug2), count in drug_cooccurrence.items():
        support = count / total_transactions
        cooccurrence_data.append({
            'drug1': drug1,
            'drug2': drug2,
            'cooccurrence_count': count,
            'support': support
        })
    
    cooccurrence_df = pd.DataFrame(cooccurrence_data)
    cooccurrence_df = cooccurrence_df.sort_values('support', ascending=False)
    
    print(f"    Found {len(cooccurrence_df)} drug pairs that co-occur")
    print(f"\n Top 10 Drug Co-occurrences:")
    print(f"{'Drug 1':<25} {'Drug 2':<25} {'Count':<6} {'Support'}")
    print("-" * 70)
    
    for i, (idx, row) in enumerate(cooccurrence_df.head(10).iterrows(), 1):
        print(f"{row['drug1'][:24]:<25} {row['drug2'][:24]:<25} {row['cooccurrence_count']:<6} {row['support']:.3f}")

# 4. Statistical Summary
print(f"\n Association Rule Mining Summary:")
print(f"    Total transactions analyzed: {len(condition_drug_transactions)}")
print(f"    Unique drugs: {len(te.columns_) if 'te' in locals() else 'N/A'}")
print(f"    Frequent itemsets found: {len(frequent_itemsets)}")
print(f"    Association rules generated: {len(association_rules_df)}")
print(f"    Drug co-occurrence pairs: {len(drug_cooccurrence)}")

# Store results for visualization
print(f"\n Association rule mining completed!")
print(f"    Ready for pattern visualization")
print(f"    Rules available for business insight analysis")


 FREQUENT ITEMSET MINING & ASSOCIATION RULE GENERATION

 1. Mining Frequent Drug Combinations (Apriori Algorithm)
    Minimum support threshold: 0.064
    Found 1 frequent itemsets

 Top 10 Frequent Drug Combinations:

 2. Generating Association Rules
    No association rules met the minimum confidence threshold

 3. Direct Drug Co-occurrence Analysis
    Found 178759 drug pairs that co-occur

 Top 10 Drug Co-occurrences:
Drug 1                    Drug 2                    Count  Support
----------------------------------------------------------------------
budesonide                cromolyn                  2      0.043
fluticasone               triamcinolone             2      0.043
ciclesonide               triamcinolone             2      0.043
fluticasone               mometasone                2      0.043
mometasone                triamcinolone             2      0.043
Edarbi                    Tenoretic 100             1      0.021
Edarbi                    pindolol            

---
## 4. Visualization and Business Insights


In [5]:
# Visualization and Business Insights
print("ASSOCIATION RULE VISUALIZATION & BUSINESS INSIGHTS")
print("=" * 55)

# 1. Visualize Association Rules (if available)
if len(association_rules_df) > 0:
    print("\n 1. Association Rules Visualization")
    
    # Create scatter plot of rules
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['Support vs Confidence', 'Support vs Lift', 
                       'Confidence vs Lift', 'Rule Strength Distribution'],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Plot 1: Support vs Confidence
    fig.add_trace(
        go.Scatter(x=association_rules_df['support'], 
                  y=association_rules_df['confidence'],
                  mode='markers',
                  text=association_rules_df['rule_description'],
                  name="Rules",
                  marker=dict(size=8, opacity=0.7)),
        row=1, col=1
    )
    
    # Plot 2: Support vs Lift
    fig.add_trace(
        go.Scatter(x=association_rules_df['support'], 
                  y=association_rules_df['lift'],
                  mode='markers',
                  text=association_rules_df['rule_description'],
                  name="Rules",
                  marker=dict(size=8, opacity=0.7, color='red')),
        row=1, col=2
    )
    
    # Plot 3: Confidence vs Lift
    fig.add_trace(
        go.Scatter(x=association_rules_df['confidence'], 
                  y=association_rules_df['lift'],
                  mode='markers',
                  text=association_rules_df['rule_description'],
                  name="Rules",
                  marker=dict(size=8, opacity=0.7, color='green')),
        row=2, col=1
    )
    
    # Plot 4: Lift distribution
    fig.add_trace(
        go.Histogram(x=association_rules_df['lift'], 
                    nbinsx=20,
                    name="Lift Distribution",
                    marker_color='orange'),
        row=2, col=2
    )
    
    fig.update_layout(height=800, title_text="Association Rules Analysis Dashboard", 
                     showlegend=False)
    
    # Robust display
    try:
        fig.show()
    except Exception as e:
        print(f" Interactive plot failed: {e}")
        print("Using matplotlib backup...")
        
        plt.figure(figsize=(15, 10))
        
        plt.subplot(2, 2, 1)
        plt.scatter(association_rules_df['support'], association_rules_df['confidence'], alpha=0.7)
        plt.xlabel('Support')
        plt.ylabel('Confidence')
        plt.title('Support vs Confidence')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(2, 2, 2)
        plt.scatter(association_rules_df['support'], association_rules_df['lift'], alpha=0.7, color='red')
        plt.xlabel('Support')
        plt.ylabel('Lift')
        plt.title('Support vs Lift')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(2, 2, 3)
        plt.scatter(association_rules_df['confidence'], association_rules_df['lift'], alpha=0.7, color='green')
        plt.xlabel('Confidence')
        plt.ylabel('Lift')
        plt.title('Confidence vs Lift')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(2, 2, 4)
        plt.hist(association_rules_df['lift'], bins=20, alpha=0.7, color='orange')
        plt.xlabel('Lift')
        plt.ylabel('Frequency')
        plt.title('Lift Distribution')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# 2. Drug Co-occurrence Network (if NetworkX is available)
if nx and len(drug_cooccurrence) > 20:
    print("\n 2. Drug Co-occurrence Network Analysis")
    
    # Create network graph
    G = nx.Graph()
    
    # Add edges with weights (co-occurrence counts)
    for (drug1, drug2), count in list(drug_cooccurrence.items())[:50]:  # Top 50 pairs
        if count >= 2:  # Only include pairs that co-occur at least twice
            G.add_edge(drug1, drug2, weight=count)
    
    print(f"    Network created with {G.number_of_nodes()} drugs and {G.number_of_edges()} connections")
    
    # Calculate network metrics
    if G.number_of_nodes() > 0:
        degree_centrality = nx.degree_centrality(G)
        top_central_drugs = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
        
        print(f"   🔝 Most Connected Drugs (Degree Centrality):")
        for i, (drug, centrality) in enumerate(top_central_drugs, 1):
            print(f"      {i}. {drug}: {centrality:.3f}")

# 3. Business Insights and Recommendations
print("\n 3. Business Insights and Recommendations")
print("=" * 40)

# Insight 1: Drug Substitution Recommendations
if 'cooccurrence_df' in locals() and len(cooccurrence_df) > 0:
    print(f"\n Drug Substitution Insights:")
    print(f"   Based on co-occurrence patterns, the following drug pairs are frequently")
    print(f"   used for similar conditions and could be considered as alternatives:")
    
    for i, (idx, row) in enumerate(cooccurrence_df.head(5).iterrows(), 1):
        print(f"   {i}. {row['drug1']} ↔ {row['drug2']} (co-occur in {row['cooccurrence_count']} conditions)")

# Insight 2: Treatment Complexity Analysis
if 'condition_drug_transactions' in locals():
    condition_complexity = {}
    for transaction in condition_drug_transactions:
        complexity = len(transaction)
        condition_complexity[complexity] = condition_complexity.get(complexity, 0) + 1
    
    print(f"\n Treatment Complexity Analysis:")
    print(f"   Distribution of drug options per medical condition:")
    for complexity in sorted(condition_complexity.keys()):
        count = condition_complexity[complexity]
        percentage = (count / len(condition_drug_transactions)) * 100
        print(f"   {complexity} drugs: {count} conditions ({percentage:.1f}%)")

# Insight 3: Market Opportunities
if len(association_rules_df) > 0:
    high_lift_rules = association_rules_df[association_rules_df['lift'] > 2.0]
    print(f"\n Market Opportunity Insights:")
    print(f"   Found {len(high_lift_rules)} high-lift rules (lift > 2.0)")
    print(f"   These represent strong associations that could indicate:")
    print(f"   • Drug combination opportunities")
    print(f"   • Market gaps for alternative medications")
    print(f"   • Treatment protocol optimization opportunities")

# Insight 4: Risk Assessment
if 'drug_cooccurrence' in locals():
    total_drug_pairs = len(drug_cooccurrence)
    frequent_pairs = len([pair for pair, count in drug_cooccurrence.items() if count >= 3])
    
    print(f"\n Drug Interaction Risk Assessment:")
    print(f"   Total drug pairs identified: {total_drug_pairs}")
    print(f"   Frequently co-occurring pairs: {frequent_pairs}")
    print(f"   Risk assessment recommendation: Monitor the {frequent_pairs} frequent")
    print(f"   drug combinations for potential interactions or synergistic effects")

# Summary Report
print(f"\n ASSOCIATION RULE MINING SUMMARY REPORT")
print("=" * 50)
print(f" Data Processing:")
print(f"   • Analyzed {len(condition_drug_transactions)} condition-drug transactions")
print(f"   • Processed {df['medical_condition'].nunique()} unique medical conditions")
print(f"   • Examined {df['drug_name'].nunique()} unique medications")

print(f"\n Pattern Discovery:")
if 'frequent_itemsets' in locals():
    print(f"   • Discovered {len(frequent_itemsets)} frequent drug combinations")
print(f"   • Generated {len(association_rules_df)} association rules")
print(f"   • Identified {len(drug_cooccurrence)} drug co-occurrence patterns")

print(f"\n Business Value:")
print(f"   • Drug recommendation system foundation established")
print(f"   • Alternative medication pathways identified")
print(f"   • Treatment complexity patterns revealed")
print(f"   • Market opportunity insights generated")

print(f"\n Next Steps:")
print(f"   1. Integrate findings into clinical decision support systems")
print(f"   2. Validate associations with clinical experts")
print(f"   3. Develop automated drug recommendation algorithms")
print(f"   4. Create real-time monitoring for drug interaction patterns")

print(f"\n Association Rule Mining Analysis Complete!")
print("   Ready to proceed with machine learning modeling phase")


ASSOCIATION RULE VISUALIZATION & BUSINESS INSIGHTS

 2. Drug Co-occurrence Network Analysis
    Network created with 0 drugs and 0 connections

 3. Business Insights and Recommendations

 Drug Substitution Insights:
   Based on co-occurrence patterns, the following drug pairs are frequently
   used for similar conditions and could be considered as alternatives:
   1. budesonide ↔ cromolyn (co-occur in 2 conditions)
   2. fluticasone ↔ triamcinolone (co-occur in 2 conditions)
   3. ciclesonide ↔ triamcinolone (co-occur in 2 conditions)
   4. fluticasone ↔ mometasone (co-occur in 2 conditions)
   5. mometasone ↔ triamcinolone (co-occur in 2 conditions)

 Treatment Complexity Analysis:
   Distribution of drug options per medical condition:
   5 drugs: 2 conditions (4.3%)
   7 drugs: 2 conditions (4.3%)
   9 drugs: 2 conditions (4.3%)
   11 drugs: 1 conditions (2.1%)
   13 drugs: 1 conditions (2.1%)
   14 drugs: 1 conditions (2.1%)
   17 drugs: 1 conditions (2.1%)
   19 drugs: 1 conditions

---